# 🧪 PyTorch Lab 0: Tensors & Autograd (Basics)

Welcome! This lab will get you hands-on with **PyTorch**: creating tensors, defining functions,
using **autograd**, and manipulating gradients.

Go to https://pytorch.org/get-started/locally/, select your config and package manager and install.

**What you'll learn**:
- Creating and inspecting tensors
- Defining scalar & multivariate functions
- Backpropagation via `autograd`
- Zeroing/accumulating gradients and `detach()`
- (Bonus) Finite-difference gradient check

## 0. Setup

In [ ]:
import torch
print('PyTorch version:', torch.__version__)
torch.manual_seed(0)

## 1. Tensors Basics

A **tensor** is a generalization of vectors and matrices to potentially higher dimensions.

In [ ]:
# Create basic tensors
x = torch.tensor([2.0], requires_grad=True)
y = torch.tensor([3.0], requires_grad=True)

print('x =', x)
print('y =', y)
z = x + y
print('z = x + y =', z)
print('z.shape =', z.shape)

### Exercise 1.1 — Create a 3×3 random matrix

- Create a tensor `A` of shape **(3, 3)** with random values.
- Compute its transpose `A_T` and verify the shape.

In [ ]:
# TODO: create A and A_T
# Hint: use torch.randn
A = torch.randn(3, 3)
A_T = A.t()

# ===== quick checks =====
assert A.shape == (3, 3)
assert A_T.shape == (3, 3)
print('A =\n', A)
print('A_T =\n', A_T)

## 2. Defining a Function

Start with a simple scalar function and build intuition.

In [ ]:
# f(x) = x^2 at x=4
x = torch.tensor([4.0], requires_grad=True)
f = x**2
print('f(x) =', f.item())

### Exercise 2.1 — Polynomial

Define and evaluate the polynomial:  
\( f(x) = 3x^3 + 2x^2 + 5 \) at \( x = 2 \).

In [ ]:
# TODO: define poly at x=2 with requires_grad=True
x = torch.tensor([2.0], requires_grad=True)
f = 3 * x**3 + 2 * x**2 + 5
print('f(2) =', f.item())

## 3. Using Autograd

Call `.backward()` on a scalar output to compute gradients of inputs marked with `requires_grad=True`.

In [ ]:
# Gradient of f(x) = x^2 at x=4
x = torch.tensor([4.0], requires_grad=True)
f = x**2
f.backward()
print('df/dx at x=4 =', x.grad.item())  # Expected 8

### Exercise 3.1 — Autograd on the polynomial

Compute the gradient of \( f(x) = 3x^3 + 2x^2 + 5 \) at \( x = 2 \).  
*(Analytical answer for comparison: \( f'(x) = 9x^2 + 4x \) so \( f'(2) = 9*4 + 8 = 44 \)).

In [ ]:
# TODO: compute gradient with autograd
x = torch.tensor([2.0], requires_grad=True)
f = 3 * x**3 + 2 * x**2 + 5
f.backward()
print('df/dx at x=2 =', x.grad.item())
assert abs(x.grad.item() - 44.0) < 1e-5

## 4. Gradient Manipulation

Gradients **accumulate** by default on leaf tensors. Use `zero_()` to reset between backward passes.

In [ ]:
x = torch.tensor([2.0], requires_grad=True)

f1 = 3*x**3 + 2*x**2 + 5
f1.backward()
print('After first backward, grad =', x.grad.item())

# Zero gradients before next backward
x.grad.zero_()

f2 = (x + 1)**2
f2.backward()
print('After zero + second backward, grad =', x.grad.item())

### Exercise 4.1 — Observe accumulation

Call `.backward()` twice **without** zeroing gradients. What do you observe?

In [ ]:
x = torch.tensor([2.0], requires_grad=True)

# Calcul de la fonction f = (x + 1)^2
f = (x + 1)**2
f.backward()
print('grad after first backward =', x.grad.item())

# Appel backward() une deuxième fois sans remettre à zéro les gradients
f = (x + 1)**2
f.backward()
print('grad after second backward (no zero_) =', x.grad.item())

# Commentaire d'observation:
# Les gradients s'accumulent naturellement dans x.grad.
# Après deux backward(), le gradient affiché est le double de celui après un seul backward.


## 5. Stopping Gradient Flow with `detach()`

Use `detach()` to stop autograd from tracking a tensor in the graph.

In [ ]:
x = torch.tensor([2.0], requires_grad=True)
y = x**2
z = y.detach()
print('y requires_grad:', y.requires_grad)
print('z requires_grad:', z.requires_grad)

## 6. Multivariate Functions

Autograd handles multiple inputs naturally.

In [ ]:
x = torch.tensor(2.0, requires_grad=True)
y = torch.tensor(3.0, requires_grad=True)

f = x*y + y**2
f.backward()

print('df/dx =', x.grad.item())  # expected: y => 3
print('df/dy =', y.grad.item())  # expected: x + 2y => 2 + 6 = 8

## 🔬 Bonus: Finite-Difference Gradient Check (Optional)

Use finite differences to **numerically** approximate the gradient and compare with autograd.

In [ ]:
def finite_diff_grad_scalar(fn, x0: float, eps: float = 1e-6):
    """Return numerical derivative of a scalar function f(x) at x0."""
    return (fn(x0 + eps) - fn(x0 - eps)) / (2 * eps)

# Compare on f(x) = 3x^3 + 2x^2 + 5 at x0 = 2
def f_np(x):
    return 3*x**3 + 2*x**2 + 5

x0 = 2.0
g_num = finite_diff_grad_scalar(f_np, x0)
print('finite-diff grad ≈', g_num)

# Autograd grad for comparison
x = torch.tensor([x0], requires_grad=True)
f = 3*x**3 + 2*x**2 + 5
f.backward()
print('autograd grad =', x.grad.item())